In [5]:
import open3d as o3d
import torch
import laspy
import numpy as np
import matplotlib.pyplot as plt

from laspy import LasData
from pathlib import Path
from tqdm import tqdm
from torch.cuda import amp
from torchsparse.utils.quantize import sparse_quantize
from torchsparse import SparseTensor

from EHydro_TreeUnet import TreeProjector

In [6]:
SAVE_CHUNKS = True
USE_STORED = True
SAVE_SEGMENTED = True
SAVE_ALL_SEGMENTS = True
STEP = 50
MIN_POINTS_PER_PC = 10000
WEIGHTS_FILE = 'tree_projector_weights_x16_0.2.pth'
CHANNELS = [16, 32, 64, 128]
LATENT_DIM = 256
MAX_INSTANCES = 64
VOXEL_SIZE = 0.2

class_names = ['Terrain', 'Low Vegetation', 'Stem', 'Canopy']
class_colormap = np.array([
    [128, 128, 128], # clase 0 - Terrain - gris
    [147, 255, 138], # clase 1 - Low vegetation - verde claro
    [255, 165, 0],   # clase 2 - Stem - naranja
    [0, 128, 0],     # clase 3 - Canopy - verde oscuro
], dtype=np.uint8)

pointcloud_folder = Path('/mnt/c/Users/samue/Desktop/PointClouds')
input_folder = pointcloud_folder / 'input'
output_folder = pointcloud_folder / 'output'
segmented_folder = pointcloud_folder / 'segmented'
segmented_by_folder = pointcloud_folder / 'segmented/by_folders'

input_folder.mkdir(parents=True, exist_ok=True)
output_folder.mkdir(parents=True, exist_ok=True)
segmented_folder.mkdir(parents=True, exist_ok=True)
segmented_by_folder.mkdir(parents=True, exist_ok=True)

In [7]:
def chunkerize(file: LasData, ext: str):
    points = file.points
    coords = np.vstack((file.x, file.y)).transpose()

    idx  = np.floor_divide(coords, STEP).astype(int)
    idx -= idx.min(axis=0)

    idx = np.ravel_multi_index(idx.T, idx.max(axis=0) + 1)
    chunks = []
    for i, unique_idx in enumerate(tqdm(np.unique(idx))):
        chunk_points = points[idx == unique_idx]
        if len(chunk_points) < MIN_POINTS_PER_PC:
            continue

        chunk_file = laspy.LasData(file.header)
        chunk_file.points = chunk_points
        chunks.append(chunk_file)
        
        if SAVE_CHUNKS:
            chunk_file.write(output_folder / f'plot_{i}{ext}')

    return chunks


In [8]:
extensions = ('.laz', '.las')
point_clouds = sorted(
            [f for f in input_folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions],
            key=lambda f: f.name
        )

model = TreeProjector(1, len(class_names), MAX_INSTANCES, channels = CHANNELS, latent_dim = LATENT_DIM)
model.load_state_dict(torch.load(Path('./weights') / WEIGHTS_FILE))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

pcd = o3d.geometry.PointCloud()
for point_cloud in point_clouds:
    if USE_STORED:
        chunks = [laspy.read(f) for f in output_folder.rglob("*") if f.is_file() and f.suffix.lower() in extensions]
    else:
        chunks = chunkerize(laspy.read(point_cloud), point_cloud.suffix.lower())

    segmented_chunks = []

    for i, chunk in enumerate(tqdm(chunks)):
        coords = np.vstack((chunk.x, chunk.y, chunk.z)).transpose()
        min_coords = coords.min(axis=0)
        coords -= min_coords

        intensity = np.array(chunk.intensity)[:, None]
        min_intensity = np.min(intensity)
        max_intensity = np.max(intensity)
        i_norm = (intensity - min_intensity) / (max_intensity - min_intensity)

        voxels, indices, inverse_map = sparse_quantize(coords, VOXEL_SIZE, return_index=True, return_inverse=True)

        voxels = torch.tensor(voxels, dtype=torch.int).to(device)
        batch_index = torch.zeros((voxels.shape[0], 1), dtype=torch.int, device=voxels.device)
        voxels = torch.cat([batch_index, voxels], dim=1)
        feat = torch.tensor(i_norm.astype(np.float32), dtype=torch.float).to(device)

        inputs = SparseTensor(coords=voxels, feats=feat)
        with amp.autocast(enabled=True):
            semantic_output, instance_output = model(inputs)

        voxels = semantic_output.C.cpu().numpy()
        semantic_output = torch.argmax(semantic_output.F.cpu(), dim=1).numpy()
        instance_output = torch.argmax(instance_output.F.cpu(), dim=1).numpy()
        instance_output[(semantic_output == 0) | (semantic_output == 1)] = -1

        semantic_output = semantic_output[inverse_map]
        instance_output = (instance_output + (64 * i))[inverse_map]

        out_file = laspy.LasData(header=chunk.header, points=chunk.points.copy())
        out_file.add_extra_dims([laspy.ExtraBytesParams(name="semantic_pred", type=np.int16), laspy.ExtraBytesParams(name="instance_pred", type=np.int32)])
        out_file.semantic_pred = semantic_output
        out_file.instance_pred = instance_output

        segmented_chunks.append(out_file)
        if SAVE_ALL_SEGMENTS:
            out_file.write(segmented_by_folder / f'plot_{i}.las')

    with laspy.open(segmented_folder / 'combined.las', mode='w', header=chunks[0].header) as file:
        for chunk in segmented_chunks:
            file.write_points(chunk.points)


100%|██████████| 106/106 [00:29<00:00,  3.63it/s]
